# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
filepath

'/home/workspace/event_data'

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

'''
myDialect is working to set the formatting, quoted 'ALL', every element will be quoted ''
'''
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

'''
Next function creates a new CSV file with the specified format and columns
'''

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    '''
    event data is created to extract the data and format the 
    way needed to import in the database write.writerrow is 
    defining the columns required plus an interaction for each 
    row0, row2 by extracting the specific data and putting 
    on the formatting defined by column names
    '''

    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    '''
    Next loop does loop thru each row, extract the content for
    row0 (defined column name as artist),
    row2 (defined column name as firstName), 
    row4... 
    '''
    
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
#Reading CSV file
df = pd.read_csv("event_datafile_new.csv")

In [7]:
#Checking for missing values and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [8]:
#Checking first few rows
df.head(5)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [9]:
#Checking last 5 rows
df.tail()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
6815,Sidewalk Prophets,Molly,F,0,Taylor,260.62322,free,"St. Louis, MO-IL",464,You Love Me Anyway (Album),35
6816,Rise Against,Molly,F,1,Taylor,221.17832,free,"St. Louis, MO-IL",464,Torches,35
6817,K'Naan,Molly,F,2,Taylor,220.49914,free,"St. Louis, MO-IL",464,Wavin' Flag,35
6818,Patrick Jumpen,Ryan,M,0,Smith,208.87465,free,"San Jose-Sunnyvale-Santa Clara, CA",480,Holiday,26
6819,Alicia Keys,Ryan,M,1,Smith,216.47628,free,"San Jose-Sunnyvale-Santa Clara, CA",480,Empire State Of Mind (Part II) Broken Down,26


In [10]:
#Sampling 5 records
df.sample(5)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
1155,Eminem,Harper,M,12,Barrett,284.18567,paid,"New York-Newark-Jersey City, NY-NJ-PA",129,The Real Slim Shady,42
6537,Red Hot Chili Peppers,Hayden,F,14,Brock,334.65424,paid,"Detroit-Warren-Dearborn, MI",117,Snow [Hey Oh] (Album Version),72
2832,Silverchair,Avery,F,41,Watkins,319.13751,paid,"San Jose-Sunnyvale-Santa Clara, CA",324,Without You (Album Version),30
5543,The Antlers,Aiden,M,4,Hess,328.88118,free,"La Crosse-Onalaska, WI-MN",986,Epilogue,86
334,Sparks,Aleena,F,38,Kirby,199.02649,paid,"Waterloo-Cedar Falls, IA",196,Lost And Found,44


In [11]:
#Checking some descriptive statistics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
itemInSession,6820.0,22.761144,23.444636,0.00000,4.000000,15.000000,35.000000,126.00000
length,6820.0,247.032221,102.975921,15.85587,197.321998,232.972605,274.121992,2594.87302
sessionId,6820.0,599.181818,284.953333,3.00000,374.000000,605.000000,834.000000,1114.00000
userId,6820.0,54.681232,28.162734,2.00000,29.000000,49.000000,80.000000,101.00000


In [12]:
#Checking results for question 1 using Pandas to double check response later
df.loc[df['sessionId']==338].sort_values(by=["artist"], ascending=True).head(30)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
444,Faithless,Ava,F,4,Robinson,495.30730,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50
443,Mike And The Mechanics,Ava,F,3,Robinson,275.12118,free,"New Haven-Milford, CT",338,A Beggar On A Beach Of Gold,50
441,Pixies,Ava,F,1,Robinson,89.36444,free,"New Haven-Milford, CT",338,Build High,50
442,The Roots / Jack Davey,Ava,F,2,Robinson,155.95057,free,"New Haven-Milford, CT",338,Atonement,50


In [13]:
#Checking results for question 3 using Pandas to double check response later
df.loc[df['song']=='All Hands Against His Own'].sort_values(by=["firstName"], ascending=True).head(10)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80


In [14]:
#Searching for duplicated values
duplicated = df.duplicated()
sum(duplicated)

0

#### Creating Cluster

In [15]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [16]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2
    WITH REPLICATION =
    {'class':'SimpleStrategy','replication_factor':1}"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [17]:
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

## Questions to be answered from the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>SOLUTION<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [18]:
#Creating table
query = "CREATE TABLE IF NOT EXISTS artist_song_title_library"
query = query + "(itemInSession int, sessionId int, artist text, song text, lenght float, PRIMARY KEY (itemInSession, sessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

In [19]:
#Loading data to the table
file = 'event_datafile_new.csv'

'''
Opening CSV files and iterating thru each line
'''
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

#Insert statement to iterate by rows
        query = "INSERT INTO artist_song_title_library (itemInSession, sessionId, artist, song, lenght)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], float(line[5])))

In [20]:
#Quering data to answer question
query = "SELECT * from artist_song_title_library WHERE ITEMINSESSION=4 AND SESSIONID = 338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.lenght)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### -------------------------------------------------------------------------------------------------------------------------------------------------

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [21]:
#Creating table
query = "CREATE TABLE IF NOT EXISTS artist_song_user_library"
query = query + "(userId int, sessionId int, artist text, song text, itemInSession int, firstName text, lastName text, PRIMARY KEY ((userId, sessionid), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)    


In [22]:
#Loading data to the table
file = 'event_datafile_new.csv'

'''
Opening CSV files and iterating thru each line
'''
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
#Insert statement to iterate by rows
        query = "INSERT INTO artist_song_user_library (userId,sessionId,artist,song,itemInSession,firstName,lastName)"
        query = query + "VALUES (%s, %s,%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

In [23]:
#Quering data to answer question
query = "SELECT artist,song,itemInSession FROM artist_song_user_library WHERE userId=10 AND sessionId = 182 ORDER BY itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song)

Down To The Bone Keep On Keepin' On
Three Drives Greece 2000
Sebastien Tellier Kilometer
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


### -------------------------------------------------------------------------------------------------------------------------------------------------

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [24]:
#Creating table
query = "CREATE TABLE IF NOT EXISTS user_song_library"
query = query + "(firstName text, lastName text, song text, userId int, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [25]:
#Loading data to the table
file = 'event_datafile_new.csv'

'''
Opening CSV files and iterating thru each line
'''
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

#Insert statement to iterate by rows
        query = "INSERT INTO user_song_library (firstName, lastName, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9],int(line[10])))

In [26]:
#Quering data to answer question
query = "SELECT firstName, lastName FROM user_song_library WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [27]:
query = "DROP TABLE artist_song_title_library;"
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [28]:
query = "DROP TABLE artist_song_user_library;"
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [29]:
query = "DROP TABLE user_song_library;"
try:
    session.execute(query)
except Exception as e:
    print(e) 

### Close the session and cluster connection¶

In [30]:
session.shutdown()
cluster.shutdown()